In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/amazon_cells_labelled.csv
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/amazon_cells_labelled.txt
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/imdb_labelled.csv
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/imdb_labelled.txt
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/readme.txt
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/yelp_labelled.csv
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/yelp_labelled.txt
/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/sentiment labelled sentences/amazon_cells_labelled.csv
/media/ghost/New V

/media/ghost/71f17ea4-f157-44e3-8ec8-4d543ed39935/home/gh05t/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/media/ghost/71f17ea4-f157-44e3-8ec8-4d543ed39935/home/gh05t/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/media/ghost/71f17ea4-f157-44e3-8ec8-4d543ed39935/home/gh05t/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'

In [2]:
amazon_df=pd.read_csv("/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/amazon_cells_labelled.txt",delimiter='\t',
                        header=None, 
                        names=['review', 'sentiment'])

imdb_df = pd.read_csv("/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/imdb_labelled.txt", 
                        delimiter='\t', 
                        header=None, 
                        names=['review', 'sentiment'])

yelp_df = pd.read_csv("/media/ghost/New Volume/dataset/sentiment-labelled-sentences-data-set/sentiment labelled sentences/yelp_labelled.txt", 
                        delimiter='\t', 
                        header=None, 
                        names=['review', 'sentiment'])



In [3]:
amazon_df.head()

,review,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [4]:
imdb_df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [5]:
yelp_df.head()

,review,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [6]:
import sys

from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from tensorflow.keras import initializers, regularizers, constraints
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV


Using TensorFlow backend.


In [7]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        #super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [8]:
data=pd.concat([amazon_df,yelp_df,imdb_df])

In [9]:
data.reset_index(drop='True',inplace=True)

In [10]:
data

,review,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
5,I have to jiggle the plug to get it to line up...,0
6,If you have several dozen or several hundred c...,0
7,If you are Razr owner...you must have this!,1
8,"Needless to say, I wasted my money.",0
9,What a waste of money and time!.,0


In [11]:
#Extracting Reviews and Sentiments
sentences=data['review'].tolist()
label=data['sentiment'].tolist()

In [12]:
# print some examples of sentences and labels

print("Sentences")
for i in range(10):
    #print(sentences[i],end="\n")
    print("{} {}".format(sentences[i],label[i]))
    


Sentences
So there is no way for me to plug it in here in the US unless I go by a converter. 0
Good case, Excellent value. 1
Great for the jawbone. 1
Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!! 0
The mic is great. 1
I have to jiggle the plug to get it to line up right to get decent volume. 0
If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one. 0
If you are Razr owner...you must have this! 1
Needless to say, I wasted my money. 0
What a waste of money and time!. 0


In [13]:
#pip install -U "tensorflow==2.0" 

Requirement already up-to-date: tensorflow==2.0 in /media/ghost/71f17ea4-f157-44e3-8ec8-4d543ed39935/home/gh05t/lib/python3.7/site-packages (2.0.0)
Note: you may need to restart the kernel to use updated packages.


In [14]:
#Create A Subword Datasets
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size=1000
tokenizer=tfds.features.text.SubwordTextEncoder.build_from_corpus(sentences,vocab_size,max_subword_length=5)

In [15]:
print("vocab size is",vocab_size)

#check the tokenizer words
num=1
print(sentences[num])
encoded_sentence=tokenizer.encode(sentences[num])
print(encoded_sentence)

vocab size is 1000
Good case, Excellent value.
[586, 485, 534, 2, 811, 862, 514, 678, 428, 165, 843, 788]


In [16]:
for i in encoded_sentence:
    print(tokenizer.decode([i]))

Go
od 
case
, 
E
x
cell
ent 
va
lu
e
.


In [17]:
for i,sent in enumerate(sentences):
    sentences[i]=tokenizer.encode(sent)

In [18]:
#print some encoded text
for i in range(10):
    print(sentences[i],end="\n")

[346, 774, 164, 774, 7, 217, 539, 23, 141, 9, 854, 165, 36, 18, 19, 281, 19, 1, 827, 825, 774, 56, 488, 6, 292, 192, 5, 119, 124, 236, 788]
[586, 485, 534, 2, 811, 862, 514, 678, 428, 165, 843, 788]
[305, 774, 23, 1, 848, 219, 104, 49, 788]
[826, 239, 14, 9, 344, 93, 23, 119, 124, 65, 405, 3, 47, 110, 20, 271, 215, 794, 738, 505, 126, 788, 819, 807, 816, 821, 824, 774, 822, 824, 821, 808, 818, 811, 819, 825, 740]
[15, 384, 362, 7, 51, 788]
[6, 43, 9, 848, 259, 415, 17, 1, 854, 165, 36, 9, 226, 18, 9, 604, 774, 254, 569, 774, 9, 226, 55, 37, 145, 152, 165, 111, 788]
[373, 48, 43, 39, 686, 52, 310, 500, 30, 105, 39, 686, 52, 394, 218, 298, 774, 533, 173, 857, 2, 689, 774, 205, 559, 459, 774, 1, 395, 774, 12, 692, 629, 774, 177, 250, 12, 668, 63, 192, 527, 788]
[373, 48, 41, 824, 839, 864, 25, 671, 84, 640, 48, 437, 8, 43, 544, 857, 775]
[820, 240, 842, 488, 9, 65, 863, 2, 6, 480, 14, 32, 674, 788]
[423, 54, 5, 480, 774, 12, 674, 774, 4, 184, 611]


In [19]:
import numpy as np
max_length =50
trunc_type='post'
padding_type='post'

#pad all Sequence

sequence_added=pad_sequences(sentences,maxlen=max_length,padding =padding_type,truncating=trunc_type)

In [20]:
#Separate the separate and Sentences on training and test data sets

training_size=int(len(sentences)*0.8)
train_seq=sequence_added[:training_size]
train_labels=label[:training_size]

test_seq=sequence_added[training_size:]
test_labels=label[training_size:]

train_labels=np.array(train_labels)
test_labels=np.array(test_labels)

In [21]:
print("Total no of Training Sequence are",len(train_seq))
print("Total no of Test Sequence are",len(test_seq))

Total no of Training Sequence are 2198
Total no of Test Sequence are 550


In [22]:
#Create a Model
embedding_dim=16
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)))
model.add(AttentionWithContext())
model.add(tf.keras.layers.Dense(6,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))




TypeError: The added layer must be an instance of class Layer. Found: <__main__.AttentionWithContext object at 0x7fb24ed22828>

In [ ]:
#fit a model
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
history=model.fit(train_seq,train_labels,epochs=50,validation_data=(test_seq,test_labels))

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
        plt.plot(history.history[string])
        plt.plot(history.history['val_'+string])
        plt.xlabel("Epochs")
        plt.ylabel(string)
        plt.legend([string, 'val_'+string])
        plt.show()

plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
def predict_review(model, new_sentences, maxlen=max_length, show_padded_sequence=True ):
    # Keep the original sentences so that we can keep using them later
    # Create an array to hold the encoded sequences
    new_sequences = []

    # Convert the new reviews to sequences
    for i, frvw in enumerate(new_sentences):
        new_sequences.append(tokenizer.encode(frvw))

    trunc_type='post' 
    padding_type='post'

    # Pad all sequences for the new reviews
    new_reviews_padded = pad_sequences(new_sequences, maxlen=max_length, 
                                 padding=padding_type, truncating=trunc_type)             

    classes = model.predict(new_reviews_padded)

    # The closer the class is to 1, the more positive the review is
    for x in range(len(new_sentences)):

        # We can see the padded sequence if desired
        # Print the sequence
        if (show_padded_sequence):
              print(new_reviews_padded[x])
        # Print the review as text
        print(new_sentences[x])
        # Print its predicted class
        print(classes[x])
        print("\n")

In [ ]:
# Use the model to predict some reviews   
fake_reviews = ["I love this phone", 
                "Everything was cold",
                "Everything was hot exactly as I wanted", 
                "Everything was green", 
                "the host seated us immediately",
                "they gave us free chocolate cake", 
                "we couldn't hear each other talk because of the shouting in the kitchen"
              ]

predict_review(model, fake_reviews)